In [32]:
import pandas as pd
from conversation_converter import procesar_conversaciones

# Movies

## Users

In [13]:
user_data_movie = pd.read_json('Movies/user_ids.json', orient='index')
user_data_movie.reset_index(inplace=True)
user_data_movie.columns = ['user_id', 'user_number']
print(user_data_movie.head())

          user_id  user_number
0  A30Q8X8B1S3GGT            0
1  A2NBOL825B93OM            1
2  A1HO9J4DCQDGP9            2
3  A1EMDSTJDUE6B0            3
4  A23QOAXJSWIBS6            4


In [14]:
user_data_movie.describe()

,user_number
count,3131.000000
mean,1565.000000
std,903.986172
min,0.000000
25%,782.500000
50%,1565.000000
75%,2347.500000
max,3130.000000


## Items

In [15]:
item_data_movie = pd.read_json('Movies/item_map.json', orient='index')
item_data_movie.reset_index(inplace=True)
item_data_movie.columns = ['item_id', 'item_name']
print(item_data_movie.head())

      item_id                                       item_name
0  630352138X                       No Highway In The Sky VHS
1  630434063X                                     Outland VHS
2  B00006SFIW                                 The Black Raven
3  6303142346  La Bella y La Bestia Beauty and the Beast  VHS
4  B001UHOWX8                                 White Christmas


In [16]:
item_data_movie.describe()

,item_id,item_name
count,9687,9687
unique,9687,9518
top,630352138X,Journey to the Center of the Earth
freq,1,4


## Conversation

In [39]:
input_file = 'Movies/Conversation.txt'
output_file = 'Movies/Conversation.json'
procesar_conversaciones(input_file, output_file)

Archivo procesado y guardado como: Movies/Conversation.json
